Caio Moreno - Deep Seek - Demo

# Serve DeepSeek R1 (Distilled Llama 8B) using provisioned throughput

This notebook demonstrates how to download and register the DeepSeek R1 distilled Llama model in Unity Catalog and deploy it using a Foundation Model APIs provisioned throughput endpoint.

## Install the `transformers` library from HuggingFace

In [0]:
!pip install transformers==4.44.2 mlflow
%restart_python

  Obtaining dependency information for transformers==4.44.2 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/43.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
  Obtaining dependency information for mlflow from https://files.pythonhosted.org/packages/e7/b7/f8d41dafefb11a58ee88082ebfe0bc8dab17f293609f8b546d86168ec934/mlflow-2.20.1-py3-none-any.whl.metadata
  Obtaining dependency information for mlflow-skinny==2.20.1 from https://files.pythonhosted.org/packages/b4/43/4e633b6141930c8dffb72ef597e9ad687b5cf5b443d6c924520850129721/mlflow_skinny-2.20.1-py3-none-any.whl.metadata
  Obtaining dependency information for alembic!=1.10.0,<2 from https://files.pythonhosted.org/packages/54/7e/ac0991d1745f7d755fc1cd381b3990a45b404b4d008fc75e2a983516fbfe/alembic-1.14.1-py3-none-any.whl.metadata
 

## Download DeepSeek R1 distilled Llama 8B 

The following code downloads the DeepSeek R1 distilled Llama 8B model to your local machine.

In [0]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


In [0]:
import os

LOCAL_DISK_HF = "/local_disk0/hf_cache"
os.makedirs(LOCAL_DISK_HF, exist_ok=True)
os.environ["HF_HOME"] = LOCAL_DISK_HF
os.environ["HF_DATASETS_CACHE"] = LOCAL_DISK_HF
os.environ["TRANSFORMERS_CACHE"] = LOCAL_DISK_HF

In [0]:
from huggingface_hub import snapshot_download
snapshot_download(model_id)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

figures/benchmark.jpg:   0%|          | 0.00/777k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

'/local_disk0/hf_cache/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161'

## Register the downloaded model to Unity Catalog

The following code shows how to start and log a run that registers the downloaded model to Unity Catalog.

In [0]:
import mlflow
import transformers

model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
uc_model_name = "deepseek_r1_distilled_llama8b_v1"

task = "llm/v1/chat"
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

transformers_model = {"model": model, "tokenizer": tokenizer}

with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=transformers_model,
        artifact_path="model",
        task=task,
        registered_model_name=f"main.msh.{uc_model_name}",
        metadata={
            "task": task,
            "pretrained_model_name": "meta-llama/Llama-3.3-70B-Instruct",
            "databricks_model_family": "LlamaForCausalLM",
            "databricks_model_size_parameters": "8b",
        },
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ba79d03b-f6d1-43df-800c-e5db4827a1c9/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-02-01 00:57:28.352032: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-01 00:57:28.412898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

...

Successfully registered model 'main.msh.deepseek_r1_distilled_llama8b_v1'.


Uploading artifacts:   0%|          | 0/83 [00:00<?, ?it/s]

Uploading /local_disk0/repl_tmp_data/ReplId-194be-fea6a-9/tmpp4lwy7yw/model/model/model-00001-of-00066.safeten…

Uploading /local_disk0/repl_tmp_data/ReplId-194be-fea6a-9/tmpp4lwy7yw/model/model/model-00065-of-00066.safeten…

Created version '1' of model 'main.msh.deepseek_r1_distilled_llama8b_v1'.


## Create a provisioned throughput endpoint for model serving

The following code shows how to create a provisioned throughput model serving endpoint to serve the Llama 70B that you downloaded and registered to Unity Catalog.

In [0]:
from mlflow.deployments import get_deploy_client


client = get_deploy_client("databricks")


endpoint = client.create_endpoint(
    name=uc_model_name,
    config={
        "served_entities": [{
            "entity_name": f"main.msh.{uc_model_name}",
            "entity_version": model_info.registered_model_version,
             "min_provisioned_throughput": 0,
             "max_provisioned_throughput": 9500,
            "scale_to_zero_enabled": True
        }],
        "traffic_config": {
            "routes": [{
                "served_model_name": f"{uc_model_name}-{model_info.registered_model_version}",
                "traffic_percentage": 100
            }]
        }
    }
)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ba79d03b-f6d1-43df-800c-e5db4827a1c9/lib/python3.11/site-packages/mlflow/deployments/databricks/__init__.py:466: UserWarning: Passing 'name', 'config', and 'route_optimized' as separate parameters is deprecated. Please pass the full API request payload as a single dictionary in the 'config' parameter.
  warnings.warn("\n".join(warnings_list), UserWarning)
